In [46]:
import pandas as pd
import re, os
from pathlib import Path
from tqdm.auto import tqdm

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Section 1: BLS CES Total Data Processing

In [48]:
# Paths & constants
raw_bls_ces_path = Path(
    '/content/drive/MyDrive/datasets/Thesis Data/Initial Data/'
    'BLS data/BLS USA Total Data'
)
USA_LABEL = 'USA'
MONTH_ABBREVIATIONS = [
    'Jan','Feb','Mar','Apr','May','Jun',
    'Jul','Aug','Sep','Oct','Nov','Dec'
]

def find_header_row(fp: Path) -> int:
    """
    Scan the first 20 rows to locate the one whose first cell is 'Year'
    and which contains at least 3 month abbreviations
    """
    preview = pd.read_excel(fp, header=None, nrows=20)
    for idx, row in preview.iterrows():
        first = str(row.iloc[0]).strip().lower()
        months_found = sum(m in row.values for m in MONTH_ABBREVIATIONS)
        if first == 'year' and months_found >= 3:
            return idx
    raise ValueError(f"No header row found in {fp.name}")

def load_ces_file(fp: Path, metric_name: str) -> pd.DataFrame:

    header_row = find_header_row(fp)
    df = pd.read_excel(fp, header=header_row)
    df.rename(columns={df.columns[0]: 'Year'}, inplace=True)

    month_cols = [m for m in MONTH_ABBREVIATIONS if m in df.columns]
    df = df[['Year'] + month_cols]

    # Melt to long form
    df_long = (
        df
        .melt(
            id_vars='Year',
            var_name='Month',
            value_name=metric_name
        )
        .dropna(subset=[metric_name])
    )
    df_long['Date'] = pd.to_datetime(
        df_long['Year'].astype(int).astype(str) + '-' + df_long['Month']
    )
    return df_long[['Date', metric_name]]

# Gather every CES file
ces_files = sorted(raw_bls_ces_path.glob('BLS_*.xlsx'))
pattern = re.compile(r'BLS_(?P<metric>[^_]+)_(?P<sector>[^.]+)\.xlsx', re.IGNORECASE)

all_frames = []
for fp in tqdm(ces_files, desc='Reading CES files'):
    m, s = pattern.match(fp.name).groups()
    metric = m.lower()
    sector = s.lower()
    df_metric = load_ces_file(fp, metric)
    df_metric['Sector'] = sector
    df_metric['State']  = USA_LABEL
    all_frames.append(df_metric)

# Combine and date-filter
bls_ces_combined = (
    pd.concat(all_frames, ignore_index=True)
      .loc[lambda d: d['Date'] >= '2008-01-01']
      .reset_index(drop=True)
)

# Save intermediate
out_folder = '/content/drive/MyDrive/datasets/Thesis Data/Final Data'
bls_ces_combined.to_csv(f'{out_folder}/bls_total_usa.csv', index=False)
print(f"Save complete: {bls_ces_combined.shape[0]} rows")


Reading CES files:   0%|          | 0/43 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
<ipython-input-48-e69570731cd9>:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_long['Date'] = pd.to_datetime(
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarni

Save complete: 8924 rows


In [49]:
raw_ces = pd.read_csv(
    '/content/drive/MyDrive/datasets/Thesis Data/Final Data/bls_total_usa.csv',
    parse_dates=['Date']
)

id_cols = ['Date','State','Sector']
metric_cols = [c for c in raw_ces.columns if c not in id_cols]

# Melt all metrics into long form
ces_long = raw_ces.melt(
    id_vars=id_cols,
    value_vars=metric_cols,
    var_name='Metric',
    value_name='Value'
).dropna(subset=['Value'])

# Pivot so each Metric becomes its own column
ces_wide = (
    ces_long
    .pivot_table(
        index=id_cols,
        columns='Metric',
        values='Value',
        aggfunc='first'
    )
    .reset_index()
)

ces_wide = ces_wide.drop_duplicates(subset=id_cols)
ces_wide.columns.name = None

ces_output = '/content/drive/MyDrive/datasets/Thesis Data/Final Data/bls_total_usa.csv'
ces_wide.to_csv(ces_output, index=False)

print("Section 2 complete: columns =", ces_wide.columns.tolist())
print("Sample data:\n", ces_wide.head())


Section 2 complete: columns = ['Date', 'State', 'Sector', 'averagehourlyearnings', 'averageweeklyearnings', 'averageweeklyhours', 'hires', 'jobopenings', 'layoffsdischarges', 'numberofemployees', 'numberofemployeeswomen', 'quits', 'unemploymentrate']
Sample data:
         Date State              Sector  averagehourlyearnings  \
0 2008-01-01   USA  leisurehospitality                  12.62   
1 2008-01-01   USA       manufacturing                  21.69   
2 2008-01-01   USA         retailtrade                  15.09   
3 2008-01-01   USA               total                    NaN   
4 2008-01-01   USA        totalnonfarm                    NaN   

   averageweeklyearnings  averageweeklyhours   hires  jobopenings  \
0                 328.12                26.0   848.0        574.0   
1                 867.60                40.0   333.0        287.0   
2                 476.84                31.6   749.0        450.0   
3                    NaN                 NaN     NaN          NaN   

# Section 2: FRED Data Processing


In [50]:
def load_fred_series(path: Path, new_name: str) -> pd.DataFrame:

    df = pd.read_csv(path, parse_dates=['observation_date'])
    df.rename(columns={'observation_date': 'Date'}, inplace=True)
    value_col = df.columns[1]
    df.rename(columns={value_col: new_name}, inplace=True)
    return df[['Date', new_name]]

fred_folder = Path(
    '/content/drive/MyDrive/datasets/Thesis Data/Initial Data/FRED data'
)

fred_tds = load_fred_series(
    fred_folder / 'FRED_Household Debt Service Payments as a Percent of Disposable Personal Income (TDSP).csv',
    'DebtServicePct'
)
fred_pce = load_fred_series(
    fred_folder / 'FRED_Personal_Consumption_Expenditures.csv',
    'PersonalConsumptionExpenditures'
)
fred_psr = load_fred_series(
    fred_folder / 'FRED_Personal_Saving_Rate.csv',
    'PersonalSavingRate'
)
fred_cs = load_fred_series(
    fred_folder / 'FRED_University_of_Michigan_Consumer_Sentiment.csv',
    'ConsumerSentimentIndex'
)

# Sequentially merge on Date
fred_merged = (
    fred_tds
    .merge(fred_pce, on='Date', how='outer')
    .merge(fred_psr, on='Date', how='outer')
    .merge(fred_cs, on='Date', how='outer')
    .sort_values('Date')
    .reset_index(drop=True)
)

fred_merged['DebtServicePct'] = fred_merged['DebtServicePct'].ffill()

fred_merged['State']  = 'USA'
fred_merged['Sector'] = 'Aggregate'

# Export final FRED data
fred_output = '/content/drive/MyDrive/datasets/Thesis Data/Final Data/fred_merged.csv'
fred_merged.to_csv(fred_output, index=False)

print(f"FRED data merged: {fred_merged.shape[0]} records")
print("Columns:", fred_merged.columns.tolist())
fred_merged.head()


FRED data merged: 291 records
Columns: ['Date', 'DebtServicePct', 'PersonalConsumptionExpenditures', 'PersonalSavingRate', 'ConsumerSentimentIndex', 'State', 'Sector']


,Date,DebtServicePct,PersonalConsumptionExpenditures,PersonalSavingRate,ConsumerSentimentIndex,State,Sector
0,2001-01-01,12.175219,6987.4,4.5,94.7,USA,Aggregate
1,2001-02-01,12.175219,7001.8,4.6,90.6,USA,Aggregate
2,2001-03-01,12.175219,6996.7,4.9,91.5,USA,Aggregate
3,2001-04-01,12.386421,7004.2,4.8,88.4,USA,Aggregate
4,2001-05-01,12.386421,7052.8,4.3,92.0,USA,Aggregate


In [51]:
fred_merged.tail()

,Date,DebtServicePct,PersonalConsumptionExpenditures,PersonalSavingRate,ConsumerSentimentIndex,State,Sector
286,2024-11-01,11.284959,20235.1,3.7,71.8,USA,Aggregate
287,2024-12-01,11.284959,20408.1,3.3,74.0,USA,Aggregate
288,2025-01-01,11.284959,20406.7,3.9,71.7,USA,Aggregate
289,2025-02-01,11.284959,20518.9,4.1,64.7,USA,Aggregate
290,2025-03-01,11.284959,20653.3,3.9,57.0,USA,Aggregate


# Section 3: BLS LAUS Data Processing (Local Area Unemployment)


In [52]:
import glob, re

In [53]:
laus_folder = Path(
    '/content/drive/MyDrive/datasets/Thesis Data/Initial Data/BLS data/BLS State Data/BLS Local Area Unemployment data'
)
laus_paths = sorted(laus_folder.glob('BLS_LAUS_*.xlsx'))
print(f"Found {len(laus_paths)} LAUS workbooks")

def find_header_row(fp: Path) -> int:
    preview = pd.read_excel(fp, header=None, nrows=20)
    for i, row in preview.iterrows():
        if str(row.iloc[0]).strip().lower() == 'year':
            return i
    raise ValueError(f"Cannot locate 'Year' header in {fp.name}")


def load_laus_all(fp: Path) -> pd.DataFrame:
    hdr = find_header_row(fp)
    df = pd.read_excel(fp, header=hdr)

    # Clean up column names
    cols = [str(c).strip() for c in df.columns]
    df.columns = cols

    # Standardize first two cols
    df = df.rename(columns={cols[0]: 'Year', cols[1]: 'Period'})

    # Every other column is a metric
    metric_cols = [c for c in df.columns if c not in ('Year','Period')]

    # Melt into long form
    df_long = (
        df
        .melt(
            id_vars=['Year','Period'],
            value_vars=metric_cols,
            var_name='Metric',
            value_name='Value'
        )
        .dropna(subset=['Value'])
    )

    # Parse a proper Date
    df_long['Date'] = pd.to_datetime(
        df_long['Year'].astype(int).astype(str)
        + '-' + df_long['Period'],
        format='%Y-%b'
    )

    # Extract Sector & State from filename
    m = re.match(
        r'BLS_LAUS_(?:(?P<sector>[^_]+)_)?(?P<state>[^.]+)\.xlsx',
        fp.name, flags=re.IGNORECASE
    )
    sector = (m.group('sector') or 'Aggregate').lower()
    state  = m.group('state')
    df_long['State']  = state
    df_long['Sector'] = sector

    return df_long[['Date','State','Sector','Metric','Value']]


all_long = pd.concat([load_laus_all(p) for p in laus_paths], ignore_index=True)


all_long = all_long.loc[all_long['Date'] >= '2008-01-01']
laus_wide = (
    all_long
    .pivot_table(
        index=['Date','State','Sector'],
        columns='Metric',
        values='Value',
        aggfunc='first'
    )
    .reset_index()
)
laus_wide.columns.name = None


out_path = (
    '/content/drive/MyDrive/datasets/Thesis Data/Final Data/bls_laus_all_metrics.csv'
)
laus_wide.to_csv(out_path, index=False)

print(f"LAUS (all metrics) saved: {laus_wide.shape[0]} rows × {laus_wide.shape[1]} cols")


Found 51 LAUS workbooks


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

LAUS (all metrics) saved: 10557 rows × 9 cols


In [54]:
laus_wide.head()

,Date,State,Sector,employment,employment-population ratio,labor force,labor force participation rate,unemployment,unemployment rate
0,2008-01-01,Alabama,aggregate,2102623.0,58.1,2203722.0,60.9,101099.0,4.6
1,2008-01-01,Alaska,aggregate,330850.0,66.3,352768.0,70.7,21918.0,6.2
2,2008-01-01,Arizona,aggregate,2853932.0,60.8,2980006.0,63.5,126074.0,4.2
3,2008-01-01,Arkansas,aggregate,1289057.0,58.9,1358050.0,62.1,68993.0,5.1
4,2008-01-01,California,aggregate,17146553.0,61.9,18216024.0,65.8,1069471.0,5.9


# Section 4: BLS State & Area Employment, Hours, and Earnings


In [55]:
# Map month abbreviations → month number
MONTH = {m: i+1 for i, m in enumerate(
    ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
)}

def add_spaces(token: str) -> str:
    if len(token) <= 2 and token.isupper():
        return token
    return re.sub(r'(?<!^)(?=[A-Z])', ' ', token).strip()

def tidy_generic(filepath: str, prefix: str, metric_name: str) -> pd.DataFrame:

    # Derive sector & state from filename
    base = os.path.basename(filepath).replace(f'BLS_{prefix}_', '')
    sector, state = base.rsplit('_', 1)
    state = add_spaces(state.replace('.xlsx',''))

    # Find the header row (first cell == 'Year')
    raw = pd.read_excel(filepath, header=None)
    hdr = raw.index[
        raw.iloc[:,0].astype(str).str.fullmatch(r'Year', case=False)
    ][0]

    # Read with correct header, clean Year column
    df = pd.read_excel(filepath, header=hdr)
    df.rename(columns=lambda c: 'Year' if str(c).lower()=='year' else c, inplace=True)
    df = df[df['Year'].astype(str).str.isnumeric()].astype({'Year':int})

    long = (
        df
        .melt('Year', var_name='Month', value_name=metric_name)
        .dropna(subset=[metric_name])
    )
    long['Date'] = pd.to_datetime({
        'year':  long['Year'],
        'month': long['Month'].map(MONTH),
        'day':   1
    })

    long['State']  = state
    long['Sector'] = sector
    return long[['Date','Year','Month','State','Sector',metric_name]]


In [56]:
FOLDER_EMP = '/content/drive/MyDrive/datasets/Thesis Data/Initial Data/BLS data/BLS State Data/BLS State and Area Employment, Hours, and Earnings/number of employees data'

emp_files = glob.glob(os.path.join(FOLDER_EMP, 'BLS_numberofemployees_*.xlsx'))
emp_files = emp_files or glob.glob(os.path.join(FOLDER_EMP, 'BLS_numberofemployees_*.XLSX'))
print(f"Part A ▸ {len(emp_files)} files found")

employees_df = pd.concat(
    [tidy_generic(fp, 'numberofemployees',      'number_of_employees')
     for fp in emp_files],
    ignore_index=True
)
employees_df = employees_df[employees_df['Year'] >= 2008]
print('NumberOfEmployees:', employees_df.shape)
employees_df.head()


Part A ▸ 255 files found


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

NumberOfEmployees: (52785, 6)


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Date,Year,Month,State,Sector,number_of_employees
7,2008-01-01,2008,Jan,Alabama,totalnonfarm,2028.7
8,2009-01-01,2009,Jan,Alabama,totalnonfarm,1947.6
9,2010-01-01,2010,Jan,Alabama,totalnonfarm,1881.6
10,2011-01-01,2011,Jan,Alabama,totalnonfarm,1885.8
11,2012-01-01,2012,Jan,Alabama,totalnonfarm,1898.7


In [57]:
FOLDER_HRS = '/content/drive/MyDrive/datasets/Thesis Data/Initial Data/BLS data/BLS State Data/BLS State and Area Employment, Hours, and Earnings/average weekly working hours  data'

hrs_files = glob.glob(os.path.join(FOLDER_HRS, 'BLS_averageweeklyhours_*.xlsx'))
hrs_files = hrs_files or glob.glob(os.path.join(FOLDER_HRS, 'BLS_averageweeklyhours_*.XLSX'))
print(f"Part B ▸ {len(hrs_files)} files found")

hours_df = pd.concat(
    [tidy_generic(fp, 'averageweeklyhours',     'average_weekly_hours')
     for fp in hrs_files],
    ignore_index=True
)
hours_df = hours_df[hours_df['Year'] >= 2008]
print('AverageWeeklyHours:', hours_df.shape)
hours_df.head()


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Part B ▸ 147 files found


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

AverageWeeklyHours: (26145, 6)


,Date,Year,Month,State,Sector,average_weekly_hours
1,2008-01-01,2008,Jan,Alabama,totalprivate,36.0
2,2009-01-01,2009,Jan,Alabama,totalprivate,34.8
3,2010-01-01,2010,Jan,Alabama,totalprivate,34.6
4,2011-01-01,2011,Jan,Alabama,totalprivate,34.8
5,2012-01-01,2012,Jan,Alabama,totalprivate,35.5


In [58]:
FOLDER_EARN = '/content/drive/MyDrive/datasets/Thesis Data/Initial Data/BLS data/BLS State Data/BLS State and Area Employment, Hours, and Earnings/average hourly earnings data'

earn_files = glob.glob(os.path.join(FOLDER_EARN, 'BLS_averagehourlyearnings_*.xlsx'))
earn_files = earn_files or glob.glob(os.path.join(FOLDER_EARN, 'BLS_averagehourlyearnings_*.XLSX'))
print(f"Part C ▸ {len(earn_files)} files found")

earnings_df = pd.concat(
    [tidy_generic(fp, 'averagehourlyearnings', 'average_hourly_earnings')
     for fp in earn_files],
    ignore_index=True
)
earnings_df = earnings_df[earnings_df['Year'] >= 2008]
print('AverageHourlyEarnings:', earnings_df.shape)
earnings_df.head()


Part C ▸ 148 files found


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

AverageHourlyEarnings: (30636, 6)


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Date,Year,Month,State,Sector,average_hourly_earnings
1,2008-01-01,2008,Jan,Alabama,totalprivate,19.27
2,2009-01-01,2009,Jan,Alabama,totalprivate,19.82
3,2010-01-01,2010,Jan,Alabama,totalprivate,19.94
4,2011-01-01,2011,Jan,Alabama,totalprivate,19.97
5,2012-01-01,2012,Jan,Alabama,totalprivate,20.40


In [59]:
# Merge all metrics into one file
merged_df = (
    employees_df
    .merge(hours_df,    on=['Date','Year','Month','State','Sector'], how='outer')
    .merge(earnings_df, on=['Date','Year','Month','State','Sector'], how='outer')
    .sort_values(['Date','State','Sector'])
    .reset_index(drop=True)
)

# Export final combined table
OUT_FINAL = '/content/drive/MyDrive/datasets/Thesis Data/Final Data/bls_state_area_employment_hours_earnings.csv'
merged_df.to_csv(OUT_FINAL, index=False)
print(f"Final merged CSV saved → {OUT_FINAL}")
print("Shape:", merged_df.shape)
merged_df.head()


Final merged CSV saved → /content/drive/MyDrive/datasets/Thesis Data/Final Data/bls_state_area_employment_hours_earnings.csv
Shape: (52785, 8)


,Date,Year,Month,State,Sector,number_of_employees,average_weekly_hours,average_hourly_earnings
0,2008-01-01,2008,Jan,Alabama,leisurehospitality,174.1,NaN,9.91
1,2008-01-01,2008,Jan,Alabama,manufacturing,293.2,41.7,20.52
2,2008-01-01,2008,Jan,Alabama,retailtrade,241.3,NaN,NaN
3,2008-01-01,2008,Jan,Alabama,totalnonfarm,2028.7,NaN,NaN
4,2008-01-01,2008,Jan,Alabama,totalprivate,1644.0,36.0,19.27


# Section 5: BEA Personal Consumption Expenditure and Price Index Data



In [60]:
# Personal Consumption Expenditures (PCE)
file_pce = "/content/drive/MyDrive/datasets/Thesis Data/Initial Data/BEA data/BEA_Personal Consumption Expenditures by Major Type of Product and by Major Function_in_millions_of_dollars.xlsx"
df_raw_pce = pd.read_excel(file_pce, sheet_name=0, header=None)

year_row_pce = df_raw_pce.iloc[5, :]
month_row_pce = df_raw_pce.iloc[6, :]
year_row_filled_pce = year_row_pce.ffill()
month_row_filled_pce = month_row_pce.ffill()

product_types_pce = df_raw_pce.iloc[8:, 1]
data_values_pce = df_raw_pce.iloc[8:, 2:]

col_labels_pce = [
    (str(year_row_filled_pce[i]), str(month_row_filled_pce[i]).title()[:3])
    for i in range(2, len(year_row_pce))
]

records_pce = []
for i, prod in enumerate(product_types_pce):
    if pd.isna(prod):
        continue
    for j, val in enumerate(data_values_pce.iloc[i]):
        year, month = col_labels_pce[j]
        if year is None or month is None or pd.isna(val):
            continue
        try:
            value_num = float(val)
        except:
            continue
        try:
            date = pd.to_datetime(f"{year}-{month}", format="%Y-%b")
        except:
            date = None
        records_pce.append({
            "Type of Product": str(prod).strip(),
            "Year": int(year),
            "Month": month,
            "Date": date,
            "PCE": value_num
        })

df_tidy_pce = pd.DataFrame.from_records(records_pce)
df_tidy_pce = df_tidy_pce.dropna(subset=["Date"])
df_tidy_pce["Month_Num"] = df_tidy_pce["Date"].dt.month

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [61]:
# Price Indexes
file_idx = "/content/drive/MyDrive/datasets/Thesis Data/Initial Data/BEA data/BEA_Price Indexes for Personal Consumption Expenditures by Major Type of Product and by Major Function.xlsx"
df_raw_idx = pd.read_excel(file_idx, sheet_name=0, header=None)

year_row_idx = df_raw_idx.iloc[5, :]
month_row_idx = df_raw_idx.iloc[6, :]
year_row_filled_idx = year_row_idx.ffill()
month_row_filled_idx = month_row_idx.ffill()

product_types_idx = df_raw_idx.iloc[8:, 1]
data_values_idx = df_raw_idx.iloc[8:, 2:]

col_labels_idx = [
    (str(year_row_filled_idx[i]), str(month_row_filled_idx[i]).title()[:3])
    for i in range(2, len(year_row_idx))
]

records_idx = []
for i, prod in enumerate(product_types_idx):
    if pd.isna(prod):
        continue
    for j, val in enumerate(data_values_idx.iloc[i]):
        year, month = col_labels_idx[j]
        if year is None or month is None or pd.isna(val):
            continue
        try:
            value_num = float(val)
        except:
            continue
        try:
            date = pd.to_datetime(f"{year}-{month}", format="%Y-%b")
        except:
            date = None
        records_idx.append({
            "Type of Product": str(prod).strip(),
            "Year": int(year),
            "Month": month,
            "Date": date,
            "Price Index": value_num
        })

df_tidy_idx = pd.DataFrame.from_records(records_idx)
df_tidy_idx = df_tidy_idx.dropna(subset=["Date"])
df_tidy_idx["Month_Num"] = df_tidy_idx["Date"].dt.month

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [62]:
# Use inner join on both 'Type of Product' and 'Date'
merged = pd.merge(
    df_tidy_pce[["Type of Product", "Date", "Year", "Month", "Month_Num", "PCE"]],
    df_tidy_idx[["Type of Product", "Date", "Price Index"]],
    on=["Type of Product", "Date"],
    how="inner"
)

merged = merged[["Type of Product", "Date", "Year", "Month", "Month_Num", "PCE", "Price Index"]]

merged.to_csv("/content/drive/MyDrive/datasets/Thesis Data/Final Data/bea_pce_priceindex.csv", index=False)

print(merged.head(12))
print(merged.tail(12))

                            Type of Product       Date  Year Month  Month_Num  \
0   Personal consumption expenditures (PCE) 2001-01-01  2001   Jan          1   
1   Personal consumption expenditures (PCE) 2001-02-01  2001   Feb          2   
2   Personal consumption expenditures (PCE) 2001-03-01  2001   Mar          3   
3   Personal consumption expenditures (PCE) 2001-04-01  2001   Apr          4   
4   Personal consumption expenditures (PCE) 2001-05-01  2001   May          5   
5   Personal consumption expenditures (PCE) 2001-06-01  2001   Jun          6   
6   Personal consumption expenditures (PCE) 2001-07-01  2001   Jul          7   
7   Personal consumption expenditures (PCE) 2001-08-01  2001   Aug          8   
8   Personal consumption expenditures (PCE) 2001-09-01  2001   Sep          9   
9   Personal consumption expenditures (PCE) 2001-10-01  2001   Oct         10   
10  Personal consumption expenditures (PCE) 2001-11-01  2001   Nov         11   
11  Personal consumption exp